In [1]:
import torch
device = "cpu"

import matplotlib.pyplot as plt
import skimage
from scipy.stats import mode
from scipy import ndimage
import numpy as np

/home/twhit/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Load images

In [2]:
true_counts = torch.load("images/true_counts.pt").to(device)
true_fluors = torch.load("images/true_fluors.pt").to(device)
images = torch.load("images/images.pt").to(device).numpy().astype(int)
num_images = images.shape[0]
max_objects = true_fluors.shape[1]

### Otsu thresholding

In [3]:
min_distances = np.arange(1, 10, 2)
otsu_estimated_counts = torch.zeros(min_distances.shape[0], num_images).cpu()
otsu_mae = torch.zeros(min_distances.shape[0]).cpu()

for d in range(len(min_distances)):
    for i in range(num_images):
        # Threshold to get binary mask
        cells = images[i] >= skimage.filters.threshold_otsu(images[i])

        # Compute Euclidean distance based on mask
        distance = ndimage.distance_transform_edt(cells)

        # Get coordinates of local maxima
        local_max_coords = skimage.feature.peak_local_max(distance, min_distance = min_distances[d], exclude_border = False)

        # Create mask of local_max_coords
        local_max_mask = np.zeros(distance.shape, dtype = bool)
        local_max_mask[tuple(local_max_coords.T)] = True
        
        # Label the local_max_mask
        markers = skimage.measure.label(local_max_mask)
        
        # Watershed segmentation
        labels = skimage.segmentation.watershed(-distance, markers, mask = cells, connectivity = 2)

        # Extract estimated count
        num_foreground_labels = len(np.unique(labels)) - 1
        otsu_estimated_counts[d,i] = num_foreground_labels * (num_foreground_labels <= 2*max_objects) + 0 * (num_foreground_labels > 2*max_objects)
    
    otsu_mae[d] = (otsu_estimated_counts[d] - true_counts).abs().mean()

In [4]:
torch.stack((torch.tensor(min_distances).cpu(), otsu_mae))

tensor([[1.0000, 3.0000, 5.0000, 7.0000, 9.0000],
        [1.2730, 0.6600, 0.6610, 0.7460, 1.7540]])

### Li thresholding

In [5]:
min_distances = np.arange(1, 10, 2)
li_estimated_counts = torch.zeros(min_distances.shape[0], num_images).cpu()
li_mae = torch.zeros(min_distances.shape[0]).cpu()

for d in range(len(min_distances)):
    for i in range(num_images):
        # Threshold to get binary mask
        cells = images[i] >= skimage.filters.threshold_li(images[i])

        # Compute Euclidean distance based on mask
        distance = ndimage.distance_transform_edt(cells)

        # Get coordinates of local maxima
        local_max_coords = skimage.feature.peak_local_max(distance, min_distance = min_distances[d], exclude_border = False)

        # Create mask of local_max_coords
        local_max_mask = np.zeros(distance.shape, dtype = bool)
        local_max_mask[tuple(local_max_coords.T)] = True
        
        # Label the local_max_mask
        markers = skimage.measure.label(local_max_mask)
        
        # Watershed segmentation
        labels = skimage.segmentation.watershed(-distance, markers, mask = cells, connectivity = 2)

        # Extract estimated count
        num_foreground_labels = len(np.unique(labels)) - 1
        li_estimated_counts[d,i] = num_foreground_labels * (num_foreground_labels <= 2*max_objects) + 0 * (num_foreground_labels > 2*max_objects)
    
    li_mae[d] = (li_estimated_counts[d] - true_counts).abs().mean()

In [6]:
torch.stack((torch.tensor(min_distances).cpu(), li_mae))

tensor([[1.0000, 3.0000, 5.0000, 7.0000, 9.0000],
        [1.2420, 0.5790, 0.5640, 0.6260, 1.5740]])

### Triangle thresholding

In [7]:
min_distances = np.arange(1, 10, 2)
triangle_estimated_counts = torch.zeros(min_distances.shape[0], num_images).cpu()
triangle_mae = torch.zeros(min_distances.shape[0]).cpu()

for d in range(len(min_distances)):
    for i in range(num_images):
        # Threshold to get binary mask
        cells = images[i] >= skimage.filters.threshold_triangle(images[i])

        # Compute Euclidean distance based on mask
        distance = ndimage.distance_transform_edt(cells)

        # Get coordinates of local maxima
        local_max_coords = skimage.feature.peak_local_max(distance, min_distance = min_distances[d], exclude_border = False)

        # Create mask of local_max_coords
        local_max_mask = np.zeros(distance.shape, dtype = bool)
        local_max_mask[tuple(local_max_coords.T)] = True
        
        # Label the local_max_mask
        markers = skimage.measure.label(local_max_mask)
        
        # Watershed segmentation
        labels = skimage.segmentation.watershed(-distance, markers, mask = cells, connectivity = 2)

        # Extract estimated count
        num_foreground_labels = len(np.unique(labels)) - 1
        triangle_estimated_counts[d,i] = num_foreground_labels * (num_foreground_labels <= 2*max_objects) + 0 * (num_foreground_labels > 2*max_objects)
    
    triangle_mae[d] = (triangle_estimated_counts[d] - true_counts).abs().mean()

In [8]:
torch.stack((torch.tensor(min_distances).cpu(), triangle_mae))

tensor([[1.0000, 3.0000, 5.0000, 7.0000, 9.0000],
        [1.6700, 0.8590, 0.8190, 0.8370, 1.1440]])

### Write results

We'll use Li's method and set `min_distance = 5` during peak-finding since this combination achieved the lowest mean absolute error.

In [9]:
torch.save(li_estimated_counts[:,2], 'results/watershed_estimated_counts.pt')